In [ ]:
# ==============================================================================
# NOTEBOOK DE TESTES METAMÓRFICOS - NÍVEL 3: ANÁLISE DE GENERALIZAÇÃO
# ==============================================================================
# Objetivo: Avaliar a capacidade do modelo de fornecer saídas coerentes e
# fisicamente plausíveis sob condições dinâmicas e sequências de eventos complexos.
# ------------------------------------------------------------------------------

# 📦 Imports
import pandas as pd
import numpy as np
import joblib
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from pathlib import Path
from IPython.display import display

# ------------------------------------------------------------------------------
# 📁 ETAPA 1: SETUP DO AMBIENTE DE TESTE
# ------------------------------------------------------------------------------

try:
    ethanol_path = Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/data/[Etanol] Trajeto Casa-Escola-UFRN/trackLog-2023-Feb-13_06-38-49_seg.csv")
    gasoline_path = Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/data/[Gasolina] Trajeto Casa-Escola-UFRN/trackLog-2022-Dec-01_06-43-57_qui.csv")
    df_ethanol = pd.read_csv(ethanol_path)
    df_gasoline = pd.read_csv(gasoline_path)
    print("Arquivos de dados carregados com sucesso.")
except FileNotFoundError as e:
    print(f"ERRO: Arquivos de dados não encontrados. {e}")
    raise

def preparar_dados(df):
    df_copy = df.copy()
    df_copy.columns = df_copy.columns.str.strip()
    col_map = {
        'Speed (OBD)(km/h)': 'Speed(OBD)(km/h)',
        'Latitude ': 'Latitude',
        'Longitude ': 'Longitude'
    }
    df_copy.rename(columns={k: v for k, v in col_map.items() if k in df_copy.columns}, inplace=True)

    if "Speed(OBD)(km/h)" in df_copy.columns:
        speeds = df_copy["Speed(OBD)(km/h)"].fillna(0).values * 1000 / 3600
        df_copy["Acceleration"] = np.diff(speeds, prepend=speeds[0])

    return df_copy

df_ethanol = preparar_dados(df_ethanol)
df_gasoline = preparar_dados(df_gasoline)

# ------------------------------------------------------------------------------
# 🤖 Carregar modelos
# ------------------------------------------------------------------------------

try:
    modelos = {
        "ethanol": {
            "afr": joblib.load(Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/LGBMRegressor_ethanol_afr.pkl")),
            "maf": joblib.load(Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/XGBRegressor_ethanol_maf.pkl"))
        },
        "gasoline": {
            "afr": joblib.load(Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/LGBMRegressor_gasoline_afr.pkl")),
            "maf": joblib.load(Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/XGBRegressor_gasoline_maf.pkl"))
        }
    }
    print("Modelos de IA carregados com sucesso.")
except FileNotFoundError as e:
    print(f"ERRO: Arquivos de modelo não encontrados. {e}")
    raise

# ------------------------------------------------------------------------------
# 🔧 ETAPA 2: FUNÇÕES DE SUPORTE
# ------------------------------------------------------------------------------

def compute_co2(afr, maf, fuel_type):
    afr = np.array(afr)
    maf = np.array(maf)
    carbon_fraction = {"ethanol": 0.5217, "gasoline": 0.8571}
    c_fraction = carbon_fraction.get(fuel_type)
    fuel_rate = np.divide(maf, afr, out=np.zeros_like(maf, dtype=float), where=afr != 0)
    return fuel_rate * c_fraction * (44.0 / 12.0)

def encontrar_coluna(df, alternativas):
    df.columns = df.columns.str.strip()
    for alt in alternativas:
        if alt in df.columns:
            return df[alt]
    return None

def pearson_safe(x, y):
    x = np.asarray(x)
    y = np.asarray(y)
    mask = ~np.isnan(x) & ~np.isnan(y)
    if mask.sum() < 2:
        return np.nan
    try:
        return pearsonr(x[mask], y[mask])[0]
    except Exception:
        return np.nan

# ------------------------------------------------------------------------------
# 🔬 FUNÇÃO PRINCIPAL (COM PEARSON)
# ------------------------------------------------------------------------------

def executar_teste_plotly(
    df_original,
    df_modificado,
    modelo_afr,
    modelo_maf,
    fuel,
    nome_mr,
    co2_real=None
):
    features = ["Latitude", "Longitude", "Speed(OBD)(km/h)", "Acceleration"]

    for f in features:
        if f not in df_original.columns:
            df_original[f] = 0
        if f not in df_modificado.columns:
            df_modificado[f] = 0

    # ------------------------------------------------------------------
    # Predictions
    # ------------------------------------------------------------------
    co2_baseline = compute_co2(
        modelo_afr.predict(df_original[features]),
        modelo_maf.predict(df_original[features]),
        fuel
    )

    co2_metamorphic = compute_co2(
        modelo_afr.predict(df_modificado[features]),
        modelo_maf.predict(df_modificado[features]),
        fuel
    )

    # ------------------------------------------------------------------
    # Metrics
    # ------------------------------------------------------------------
    rmse = mean_squared_error(co2_baseline, co2_metamorphic, squared=False)
    pearson_baseline = pearson_safe(co2_baseline, co2_metamorphic)
    pearson_gt = (
        pearson_safe(co2_real, co2_metamorphic)
        if co2_real is not None else np.nan
    )

    # ------------------------------------------------------------------
    # Log
    # ------------------------------------------------------------------
    print("\n" + "=" * 80)
    print(f"🔁 {nome_mr} ({fuel.upper()})")
    print("Metamorphic Test – Level 3 (Generalization)")
    print(f"RMSE Baseline × Metamorphic = {rmse:.4f}")
    print(f"Pearson Baseline × Metamorphic = {pearson_baseline:.3f}")
    print(f"Pearson Ground Truth × Metamorphic = {pearson_gt:.3f}")
    print("=" * 80)

    # ------------------------------------------------------------------
    # Table (inspection)
    # ------------------------------------------------------------------
    tabela = pd.concat(
        [
            df_original[features].iloc[:10].add_suffix("_Baseline"),
            pd.DataFrame({"→": ["→"] * 10}),
            df_modificado[features].iloc[:10].add_suffix("_Metamorphic"),
            pd.DataFrame({
                "CO2_GroundTruth": co2_real[:10] if co2_real is not None else "N/A",
                "CO2_Pred_Baseline": co2_baseline[:10],
                "CO2_Pred_Metamorphic": co2_metamorphic[:10]
            })
        ],
        axis=1
    ).fillna("N/A")

    display(
        tabela.style.set_caption(
            f"Data comparison – {nome_mr} ({fuel.upper()})"
        )
    )

    # ------------------------------------------------------------------
    # Plot
    # ------------------------------------------------------------------
    fig = go.Figure()

    if co2_real is not None:
        fig.add_trace(go.Scatter(
            y=co2_real[:200],
            name="Ground Truth",
            line=dict(dash="dot", color="gray")
        ))

    fig.add_trace(go.Scatter(
        y=co2_baseline[:200],
        name="Prediction (Baseline)",
        line=dict(dash="dash", color="blue")
    ))

    fig.add_trace(go.Scatter(
        y=co2_metamorphic[:200],
        name="Prediction (Metamorphic)",
        line=dict(color="red")
    ))

    fig.update_layout(
        title=dict(
            text=(
                f"{nome_mr} – Generalization Analysis ({fuel.upper()})<br>"
                f"RMSE={rmse:.3f} | "
                f"Pearson(Baseline×Metamorphic)={pearson_baseline:.3f}"
            ),
            x=0.5
        ),
        xaxis_title="Samples",
        yaxis_title="Output Variable (E)",
        template="plotly_white"
    )

    fig.show()


# ==============================================================================
# 🔁 ETAPA 3: CASOS DE TESTE METAMÓRFICOS - NÍVEL 3
# ==============================================================================

testes_metamorficos_n3 = {
    "CT_3A_001 – Composite Scenario: Steep Uphill Simulation": lambda df: df.copy().assign(
        **{"Speed(OBD)(km/h)": 40.0, "Acceleration": 1.5}
    ),

    "CT_3B_001 – Composite Scenario: Downhill Inertia Simulation": lambda df: df.copy().assign(
        **{"Speed(OBD)(km/h)": 80.0, "Acceleration": -0.5}
    ),

    "CT_3C_001 – Composite Scenario: Hard Braking (Fuel Cut-Off)": lambda df: df.copy().assign(
        **{
            "Speed(OBD)(km/h)": (
                df["Speed(OBD)(km/h)"].shift(1, fill_value=80) - 5
            ).clip(lower=0),
            "Acceleration": -4.0
        }
    ),

    "CT_3D_001 – Composite Scenario: Constant Speed Cruise": lambda df: df.copy().assign(
        **{
            "Speed(OBD)(km/h)": 80.0,
            "Acceleration": 0.0
        }
    ),

    "CT_3E_001 – Composite Scenario: Sudden Stop After Cruise (t ≥ 50)": lambda df: (
        df.copy().assign(
            **{
                "Speed(OBD)(km/h)": df["Speed(OBD)(km/h)"].where(df.index < 50, 0.0),
                "Acceleration": df["Acceleration"].where(df.index < 50, 0.0)
            }
        )
    )
}




# ------------------------------------------------------------------------------
# 🧲 ETAPA 4: EXECUÇÃO DOS TESTES
# ------------------------------------------------------------------------------

print("=" * 80)
print("INICIANDO EXECUÇÃO DOS TESTES DE NÍVEL 3")
print("=" * 80)

afr_real_etanol = encontrar_coluna(df_ethanol, ["AirFuelRatio(Commanded)(:1)", "Air Fuel Ratio(Commanded)(:1)"])
maf_real_etanol = encontrar_coluna(df_ethanol, ["MassAirFlowRate(g/s)", "Mass Air Flow Rate(g/s)"])
co2_real_etanol = compute_co2(afr_real_etanol.values, maf_real_etanol.values, "ethanol") if afr_real_etanol is not None and maf_real_etanol is not None else None

afr_real_gasoline = encontrar_coluna(df_gasoline, ["AirFuelRatio(Commanded)(:1)", "Air Fuel Ratio(Commanded)(:1)"])
maf_real_gasoline = encontrar_coluna(df_gasoline, ["MassAirFlowRate(g/s)", "Mass Air Flow Rate(g/s)"])
co2_real_gasoline = compute_co2(afr_real_gasoline.values, maf_real_gasoline.values, "gasoline") if afr_real_gasoline is not None and maf_real_gasoline is not None else None

valores_reais = {"ethanol": co2_real_etanol, "gasoline": co2_real_gasoline}

for nome_mr, transformacao in testes_metamorficos_n3.items():
    for combustivel, df_original in [("ethanol", df_ethanol), ("gasoline", df_gasoline)]:
        df_modificado = transformacao(df_original.copy())
        executar_teste_plotly(
            df_original=df_original,
            df_modificado=df_modificado,
            modelo_afr=modelos[combustivel]["afr"],
            modelo_maf=modelos[combustivel]["maf"],
            fuel=combustivel,
            nome_mr=nome_mr,
            co2_real=valores_reais[combustivel]
        )


Arquivos de dados carregados com sucesso.
Modelos de IA carregados com sucesso.
INICIANDO EXECUÇÃO DOS TESTES DE NÍVEL 3

🔁 CT_N3_012 – Composite Scenario: Steep Uphill Simulation (ETHANOL)
Metamorphic Test – Level 3 (Generalization)
RMSE Baseline × Metamorphic = 0.5474
Pearson Baseline × Metamorphic = 0.471
Pearson Ground Truth × Metamorphic = 0.373


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,→,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,CO2_GroundTruth,CO2_Pred_Baseline,CO2_Pred_Metamorphic
0,-5.820105,-35.220818,3,0.000000,→,-5.820105,-35.220818,40.000000,1.500000,0.642839,0.742631,1.183137
1,-5.820103,-35.220828,5,0.555556,→,-5.820103,-35.220828,40.000000,1.500000,0.653249,0.771505,1.183137
2,-5.820102,-35.220838,5,0.000000,→,-5.820102,-35.220838,40.000000,1.500000,0.653249,0.771505,1.183137
3,-5.820096,-35.220843,3,-0.555556,→,-5.820096,-35.220843,40.000000,1.500000,0.679275,0.659013,1.183137
4,-5.820091,-35.220850,0,-0.833333,→,-5.820091,-35.220850,40.000000,1.500000,0.610306,0.667689,1.184775
5,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,40.000000,1.500000,0.610306,0.706858,1.184775
6,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,40.000000,1.500000,0.490587,0.706858,1.184775
7,-5.820085,-35.220856,3,0.833333,→,-5.820085,-35.220856,40.000000,1.500000,1.090483,1.137803,1.184775
8,-5.820074,-35.220859,3,0.000000,→,-5.820074,-35.220859,40.000000,1.500000,1.090483,1.137803,1.184775
9,-5.820064,-35.220858,3,0.000000,→,-5.820064,-35.220858,40.000000,1.500000,0.921315,1.137803,1.184775



🔁 CT_N3_012 – Composite Scenario: Steep Uphill Simulation (GASOLINE)
Metamorphic Test – Level 3 (Generalization)
RMSE Baseline × Metamorphic = 0.9451
Pearson Baseline × Metamorphic = 0.550
Pearson Ground Truth × Metamorphic = 0.412


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,→,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,CO2_GroundTruth,CO2_Pred_Baseline,CO2_Pred_Metamorphic
0,-5.820010,-35.220825,5,0.000000,→,-5.820010,-35.220825,40.000000,1.500000,1.002671,1.456781,2.954010
1,-5.820013,-35.220822,4,-0.277778,→,-5.820013,-35.220822,40.000000,1.500000,1.013360,1.016708,2.954010
2,-5.820016,-35.220819,5,0.277778,→,-5.820016,-35.220819,40.000000,1.500000,2.088720,1.456781,2.954010
3,-5.820014,-35.220825,7,0.555556,→,-5.820014,-35.220825,40.000000,1.500000,2.261889,1.996380,2.954010
4,-5.819992,-35.220829,9,0.555556,→,-5.819992,-35.220829,40.000000,1.500000,2.499195,2.400486,3.159595
5,-5.819974,-35.220824,12,0.833333,→,-5.819974,-35.220824,40.000000,1.500000,2.606089,2.400486,3.159595
6,-5.819944,-35.220823,15,0.833333,→,-5.819944,-35.220823,40.000000,1.500000,3.070012,2.148721,3.012688
7,-5.819908,-35.220810,18,0.833333,→,-5.819908,-35.220810,40.000000,1.500000,2.445747,1.470940,2.745029
8,-5.819866,-35.220794,18,0.000000,→,-5.819866,-35.220794,40.000000,1.500000,1.073221,1.588029,2.660020
9,-5.819821,-35.220775,16,-0.555556,→,-5.819821,-35.220775,40.000000,1.500000,0.562265,1.085921,2.704771



🔁 CT_N3_013 – Composite Scenario: Downhill Inertia Simulation (ETHANOL)
Metamorphic Test – Level 3 (Generalization)
RMSE Baseline × Metamorphic = 0.9333
Pearson Baseline × Metamorphic = 0.174
Pearson Ground Truth × Metamorphic = 0.140


...

,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,→,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,CO2_GroundTruth,CO2_Pred_Baseline,CO2_Pred_Metamorphic
0,-5.820010,-35.220825,5,0.000000,→,-5.820010,-35.220825,80.000000,0.000000,1.002671,1.456781,3.189422
1,-5.820013,-35.220822,4,-0.277778,→,-5.820013,-35.220822,80.000000,0.000000,1.013360,1.016708,3.189422
2,-5.820016,-35.220819,5,0.277778,→,-5.820016,-35.220819,80.000000,0.000000,2.088720,1.456781,3.189422
3,-5.820014,-35.220825,7,0.555556,→,-5.820014,-35.220825,80.000000,0.000000,2.261889,1.996380,3.189422
4,-5.819992,-35.220829,9,0.555556,→,-5.819992,-35.220829,80.000000,0.000000,2.499195,2.400486,2.736358
5,-5.819974,-35.220824,12,0.833333,→,-5.819974,-35.220824,80.000000,0.000000,2.606089,2.400486,2.736358
6,-5.819944,-35.220823,15,0.833333,→,-5.819944,-35.220823,80.000000,0.000000,3.070012,2.148721,2.648454
7,-5.819908,-35.220810,18,0.833333,→,-5.819908,-35.220810,80.000000,0.000000,2.445747,1.470940,2.052619
8,-5.819866,-35.220794,18,0.000000,→,-5.819866,-35.220794,80.000000,0.000000,1.073221,1.588029,2.052619
9,-5.819821,-35.220775,16,-0.555556,→,-5.819821,-35.220775,80.000000,0.000000,0.562265,1.085921,2.092170



🔁 CT_N3_016 – Composite Scenario: Sudden Stop After Cruise (t ≥ 50) (ETHANOL)
Metamorphic Test – Level 3 (Generalization)
RMSE Baseline × Metamorphic = 0.5034
Pearson Baseline × Metamorphic = 0.335
Pearson Ground Truth × Metamorphic = 0.265


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,→,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,CO2_GroundTruth,CO2_Pred_Baseline,CO2_Pred_Metamorphic
0,-5.820105,-35.220818,3,0.000000,→,-5.820105,-35.220818,3,0.000000,0.642839,0.742631,0.742631
1,-5.820103,-35.220828,5,0.555556,→,-5.820103,-35.220828,5,0.555556,0.653249,0.771505,0.771505
2,-5.820102,-35.220838,5,0.000000,→,-5.820102,-35.220838,5,0.000000,0.653249,0.771505,0.771505
3,-5.820096,-35.220843,3,-0.555556,→,-5.820096,-35.220843,3,-0.555556,0.679275,0.659013,0.659013
4,-5.820091,-35.220850,0,-0.833333,→,-5.820091,-35.220850,0,-0.833333,0.610306,0.667689,0.667689
5,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,0,0.000000,0.610306,0.706858,0.706858
6,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,0,0.000000,0.490587,0.706858,0.706858
7,-5.820085,-35.220856,3,0.833333,→,-5.820085,-35.220856,3,0.833333,1.090483,1.137803,1.137803
8,-5.820074,-35.220859,3,0.000000,→,-5.820074,-35.220859,3,0.000000,1.090483,1.137803,1.137803
9,-5.820064,-35.220858,3,0.000000,→,-5.820064,-35.220858,3,0.000000,0.921315,1.137803,1.137803



🔁 CT_N3_016 – Composite Scenario: Sudden Stop After Cruise (t ≥ 50) (GASOLINE)
Metamorphic Test – Level 3 (Generalization)
RMSE Baseline × Metamorphic = 0.8410
Pearson Baseline × Metamorphic = 0.564
Pearson Ground Truth × Metamorphic = 0.424


,Latitude_Baseline,Longitude_Baseline,Speed(OBD)(km/h)_Baseline,Acceleration_Baseline,→,Latitude_Metamorphic,Longitude_Metamorphic,Speed(OBD)(km/h)_Metamorphic,Acceleration_Metamorphic,CO2_GroundTruth,CO2_Pred_Baseline,CO2_Pred_Metamorphic
0,-5.820010,-35.220825,5,0.000000,→,-5.820010,-35.220825,5,0.000000,1.002671,1.456781,1.456781
1,-5.820013,-35.220822,4,-0.277778,→,-5.820013,-35.220822,4,-0.277778,1.013360,1.016708,1.016708
2,-5.820016,-35.220819,5,0.277778,→,-5.820016,-35.220819,5,0.277778,2.088720,1.456781,1.456781
3,-5.820014,-35.220825,7,0.555556,→,-5.820014,-35.220825,7,0.555556,2.261889,1.996380,1.996380
4,-5.819992,-35.220829,9,0.555556,→,-5.819992,-35.220829,9,0.555556,2.499195,2.400486,2.400486
5,-5.819974,-35.220824,12,0.833333,→,-5.819974,-35.220824,12,0.833333,2.606089,2.400486,2.400486
6,-5.819944,-35.220823,15,0.833333,→,-5.819944,-35.220823,15,0.833333,3.070012,2.148721,2.148721
7,-5.819908,-35.220810,18,0.833333,→,-5.819908,-35.220810,18,0.833333,2.445747,1.470940,1.470940
8,-5.819866,-35.220794,18,0.000000,→,-5.819866,-35.220794,18,0.000000,1.073221,1.588029,1.588029
9,-5.819821,-35.220775,16,-0.555556,→,-5.819821,-35.220775,16,-0.555556,0.562265,1.085921,1.085921
